# IF3170 Artificial Intelligence | Tugas Besar 2

This notebook serves as a template for the assignment. Please create a copy of this notebook to complete your work. You can add more code blocks, markdown blocks, or new sections if needed.


Group Number: 06

Group Members:
- Yusuf Ardian Sandi (13522015)
- Dhidit Abdi Aziz (13522040)
- Sa'ad Abdul Hakim (13522092)
- Rifki V. H. (13522120)

## Import Libraries

In [55]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
from IPython.display import display, Markdown
from scipy.stats import chi2_contingency, f_oneway
from sklearn.ensemble import RandomForestRegressor
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer
from sklearn.preprocessing import OneHotEncoder, StandardScaler, LabelEncoder, MinMaxScaler


# Import other libraries if needed

## Import Dataset

In [ ]:
# Example of reading a csv file from a gdrive link
# Take the file id from the gdrive file url
# https://drive.google.com/file/d/1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU/view?usp=sharing => The file id is 1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU
# and then put it in this format:
# https://drive.google.com/uc?id={file_id}
# Don't forget to change the access to public
# df = pd.read_csv('https://drive.google.com/uc?id=1ZUtiaty9RPXhpz5F2Sy3dFPHF4YIt5iU')
# df.head()

# Reading metadata
markdown_content = "### Metadata\n"
# display(Markdown(markdown_content))
# df_metadata = pd.read_csv('https://drive.google.com/uc?id=1gcHRql5AA87oOJouZ2z56d5K6nZpfRos')
df_metadata = pd.read_csv('../datasets/metadata.csv')
pd.set_option('display.max_colwidth', None)
display(df_metadata)

# Write the metadata to metadata.csv
# df_metadata.to_csv('../datasets/metadata.csv', index=False)


In [ ]:
# List of links for train_dataset
# additional_features_train.csv = https://drive.google.com/file/d/1MUHE2WwBPBUFm0G0lgCsvTBMJWWkoz13/view?usp=sharing
# basic_features_train.csv = https://drive.google.com/file/d/1zv9UMwhuBjpy6x1lrtXkhhi2UamBuxpS/view?usp=sharing
# content_features_train.csv = https://drive.google.com/file/d/1XNKxhf_DO_EMgj2c-D5vKRK0aWekPzn_/view?usp=sharing
# flow_features_train.csv = https://drive.google.com/file/d/1p7u1eY2fq_cWMvASLYdG__Uuv1JRuwa_/view?usp=sharing
# (target features) labels_train.csv = https://drive.google.com/file/d/1OL9iPj11nGyFg3HxRL0AC2iGIi1EO_lx/view?usp=sharing
# time_features_train = https://drive.google.com/file/d/18_Yxf4N2JW1iAvUJmQC7YxHAqtbF4B_4/view?usp=sharing
# (metadata) UNSW-NB15_features-modified.csv = https://drive.google.com/file/d/1gcHRql5AA87oOJouZ2z56d5K6nZpfRos/view?usp=sharing

# df_additional = pd.read_csv('../datasets/train_dataset/additional_features_train.csv')
# df_basic = pd.read_csv('../datasets/train_dataset/basic_features_train.csv')
# df_content = pd.read_csv('../datasets/train_dataset/content_features_train.csv')
# df_flow = pd.read_csv('../datasets/train_dataset/flow_features_train.csv')
# df_labels = pd.read_csv('../datasets/train_dataset/labels_train.csv')
# df_time = pd.read_csv('../datasets/train_dataset/time_features_train.csv')

# df_train = pd.merge(df_additional, df_basic, on='id')
# df_train = pd.merge(df_train, df_content, on='id')
# df_train = pd.merge(df_train, df_flow, on='id')
# df_train = pd.merge(df_train, df_labels, on='id')
# df_train = pd.merge(df_train, df_time, on='id')

# Drop label feature (QnA No two)
# df_train = df_train.drop(columns=['label'])

# Write df_train to csv
# df_train.to_csv('../datasets/train_dataset.csv', index=False)

df_train = pd.read_csv('../datasets/train_dataset.csv')

markdown_content = "#### Train Dataset\n"
display(Markdown(markdown_content))
display(df_train.head())
markdown_content = "Row Size: " + str(df_train.shape[0]) + "\n"
markdown_content += "Column Size: " + str(df_train.shape[1]) + "\n"
display(Markdown(markdown_content))


In [ ]:
# List of links for test_dataset
# additional_features_test.csv = https://drive.google.com/file/d/1p2m35E6ZfN_6ei-8LU0kbuRJuJ2m0Pwx/view?usp=sharing
# basic_features_test.csv = https://drive.google.com/file/d/18lWw3xWS2EmK6crxqZFTzWd0VmZkQTzt/view?usp=sharing
# content_features_test.csv = https://drive.google.com/file/d/1fRSZ5e-5SAg4d87smgsC4TT_OhBGi6HL/view?usp=sharing
# flow_features_test.csv = https://drive.google.com/file/d/1hUzzH8D_KyHL5HIx_9zHEZOWCkKLW_dW/view?usp=sharing
# (target features) labels_test.csv = none
# time_features_test.csv = https://drive.google.com/file/d/1S14SoGzgR8TYvV0EVWszTUoDAJqg-Ppo/view?usp=sharing
# (metadata) UNSW-NB15_features-modified.csv = https://drive.google.com/file/d/1qUWn06ET9vXBojumNI1IMKSAIuSwfaD2/view?usp=sharing

# df_additional = pd.read_csv('../datasets/test_dataset/additional_features_test.csv')
# df_basic = pd.read_csv('../datasets/test_dataset/basic_features_test.csv')
# df_content = pd.read_csv('../datasets/test_dataset/content_features_test.csv')
# df_flow = pd.read_csv('../datasets/test_dataset/flow_features_test.csv')
# df_time = pd.read_csv('../datasets/test_dataset/time_features_test.csv')

# df_test = pd.merge(df_additional, df_basic, on='id')
# df_test = pd.merge(df_test, df_content, on='id')
# df_test = pd.merge(df_test, df_flow, on='id')
# df_test = pd.merge(df_test, df_time, on='id')

# Write df_test to csv
# df_test.to_csv('../datasets/test_dataset.csv', index=False)

df_test = pd.read_csv('../datasets/test_dataset.csv')

markdown_content = "#### Test Dataset\n"
display(Markdown(markdown_content))
display(df_test.head())
markdown_content = "Row Size: " + str(df_test.shape[0]) + "\n"
markdown_content += "Column Size: " + str(df_test.shape[1]) + "\n"
display(Markdown(markdown_content))

# Exploratory Data Analysis (Optional)

Exploratory Data Analysis (EDA) is a crucial step in the data analysis process that involves examining and visualizing data sets to uncover patterns, trends, anomalies, and insights. It is the first step before applying more advanced statistical and machine learning techniques. EDA helps you to gain a deep understanding of the data you are working with, allowing you to make informed decisions and formulate hypotheses for further analysis.

In [ ]:
# Write your code here

# list of binary features
binary_features = ['is_sm_ips_ports', 'is_ftp_login']

# list of categorical 
categorical_features = df_train.select_dtypes(include=['object']).columns.tolist()
# Filter out binary features from categorical features
for feature in binary_features:
    if feature in categorical_features:
        categorical_features.remove(feature)

# list of numerical features
numerical_features = df_train.select_dtypes(include=['int64', 'float64']).columns.tolist()
# Filter out binary features from numerical features
for feature in binary_features:
    if feature in numerical_features:
        numerical_features.remove(feature)
numerical_features.remove('id')

markdown_content = "### Total Features\n"
markdown_content += f"* Total features (without id): {len(categorical_features) + len(numerical_features) + len(binary_features)}\n"
markdown_content += f"* Total categorical features: {len(categorical_features)}\n"
markdown_content += f"* Total numerical features: {len(numerical_features)}\n"
markdown_content += f"* Total binary features: {len(binary_features)}\n"
markdown_content += "\n### Categorical Features\n"
markdown_content += "\n".join([f"* {feature}" for feature in categorical_features])
markdown_content += "\n\n### Numerical Features\n"
markdown_content += "\n".join([f"* {feature}" for feature in numerical_features])
markdown_content += "\n\n### Binary Features\n"
markdown_content += "\n".join([f"* {feature}" for feature in binary_features])

display(Markdown(markdown_content))

In [ ]:
def generate_categorical_summary(data, categorical_features):
    """
    Generate markdown summary for categorical features.
    """
    markdown_content = "### Categorical Features Values\n"
    for feature in categorical_features:
        markdown_content += f"\n#### {feature}\n"
        markdown_content += f"* Unique values: {data[feature].nunique()}\n"
        markdown_content += f"* Values: {data[feature].unique()}\n"
    return markdown_content

def generate_numerical_summary(data, numerical_features):
    """
    Generate markdown summary for numerical features.
    """
    markdown_content = "### Numerical Features Values\n"
    for feature in numerical_features:
        unique_values = data[feature].nunique()
        markdown_content += f"\n#### {feature}\n"
        if unique_values > 150:  # More detailed stats for high-cardinality features
            markdown_content += f"* Min: {data[feature].min()}\n"
            markdown_content += f"* Mean: {data[feature].mean()}\n"
            markdown_content += f"* Max: {data[feature].max()}\n"
            markdown_content += f"* STD: {data[feature].std()}\n"
            markdown_content += f"* 25%: {data[feature].quantile(0.25)}\n"
            markdown_content += f"* 50%: {data[feature].quantile(0.50)}\n"
            markdown_content += f"* 75%: {data[feature].quantile(0.75)}\n"
        else:
            markdown_content += f"* Unique values: {unique_values}\n"
            markdown_content += f"* Values: {data[feature].unique()}\n"
    return markdown_content

def generate_binary_summary(data, binary_features):
    """
    Generate markdown summary for binary features.
    """
    markdown_content = "### Binary Features Values\n"
    for feature in binary_features:
        markdown_content += f"\n#### {feature}\n"
        markdown_content += f"* Unique values: {data[feature].nunique()}\n"
        markdown_content += f"* Values: {data[feature].unique()}\n"
    return markdown_content

def display_feature_summaries(data, categorical_features, numerical_features, binary_features):
    """
    Generate and display markdown summaries for categorical, numerical, and binary features.
    """
    # Generate summaries
    categorical_summary = generate_categorical_summary(data, categorical_features)
    numerical_summary = generate_numerical_summary(data, numerical_features)
    binary_summary = generate_binary_summary(data, binary_features)
    
    # Display all summaries
    display(Markdown(categorical_summary))
    display(Markdown(numerical_summary))
    display(Markdown(binary_summary))

# Call the function
display_feature_summaries(df_train, categorical_features, numerical_features, binary_features)

In [ ]:
# Code for Understanding Correlation analysis against the target feature (attack_cat)

chi_square_results = []  

# Display contingency table and Chi-Square test for each categorical feature with 'attack_cat'
for feature in categorical_features + binary_features:
    contigency_table = pd.crosstab(df_train['attack_cat'], df_train[feature])
    markdown_content = f"#### Contingency Table for {feature}\n"
    markdown_content += contigency_table.to_markdown()
    
    chi2, p, dof, expected = chi2_contingency(contigency_table)

    markdown_content += f"\n\n**Chi-Square Test Results for {feature}**:\n"
    markdown_content += f"- Chi-Square Statistic: {chi2:.4f}\n"
    markdown_content += f"- p-value: {p:.4e}\n"
    if p < 0.05:
        markdown_content += f"\n**Conclusion**: {feature} is significantly associated with 'attack_cat'.\n"
        chi_square_results.append((feature, chi2, p, True))
    else:
        markdown_content += f"\n**Conclusion**: {feature} is not significantly associated with 'attack_cat'.\n"
        chi_square_results.append((feature, chi2, p, False))
    
    display(Markdown(markdown_content))

# Visualize and compute correlation between numerical features and 'attack_cat'
fig, axes = plt.subplots(nrows=len(numerical_features), ncols=1, figsize=(15, len(numerical_features) * 5))

anova_results = []

for i, feature in enumerate(numerical_features):
    # Visualize with boxplot
    sn.boxplot(x=df_train['attack_cat'], y=df_train[feature], ax=axes[i])
    axes[i].set_title(f'Boxplot of {feature} by attack_cat')
    axes[i].set_ylabel(feature)
    axes[i].set_xlabel('attack_cat')

    groups = [group[feature].dropna() for _, group in df_train.groupby('attack_cat')]
    f_stat, p_value = f_oneway(*groups)

    markdown_content = f"#### ANOVA Results for {feature}\n"
    markdown_content += f"- F-Statistic: {f_stat:.4f}\n"
    markdown_content += f"- p-value: {p_value:.4e}\n"
    if p_value < 0.05:
        markdown_content += f"\n**  **: {feature} is significantly different across 'attack_cat' categories.\n"
        anova_results.append((feature, f_stat, p_value, True))
    else:
        markdown_content += f"\n**Conclusion**: {feature} does not differ significantly across 'attack_cat' categories.\n"
        anova_results.append((feature, f_stat, p_value, False))

    display(Markdown(markdown_content))
plt.tight_layout()
plt.show()

chi2_df = pd.DataFrame(chi_square_results, columns=['Feature', 'Chi-Square', 'p-value', 'Significant'])
markdown_content = "## Summary of Chi-Square Test Results for Categorical Features\n"
markdown_content += chi2_df.to_markdown()

anova_df = pd.DataFrame(anova_results, columns=['Feature', 'F-Statistic', 'p-value', 'Significant'])
markdown_content += "\n\n## Summary of ANOVA Results for Numerical Features\n"
markdown_content += anova_df.to_markdown()

display(Markdown(markdown_content))

In [ ]:
# Untuk melihat seberapa banyak sih missing value di binary features
# Function to plot histograms
def plot_histograms(data, features, feature_type):
    if not features:  # Check for empty feature list
        print(f"No {feature_type} features to plot.")
        return

    num_plots = len(features)
    num_cols = 3  # Number of columns for subplots
    num_rows = (num_plots + num_cols - 1) // num_cols  # Calculate number of rows

    fig, axes = plt.subplots(num_rows, num_cols, figsize=(15, num_rows * 5))
    fig.suptitle(f'Histograms of {feature_type} Features', fontsize=16)

    # Ensure axes is always 2D for consistency
    axes = axes.reshape(-1) if num_rows == 1 else axes.flat

    for i, feature in enumerate(features):
        ax = axes[i]
        if feature_type == 'Numerical':
            sn.histplot(data[feature], ax=ax, kde=True)
        else:
            sn.countplot(x=feature, data=data, ax=ax)
            ax.tick_params(axis='x', rotation=45)

        ax.set_title(feature)
        ax.set_xlabel('')  # Remove x-axis label

    # Remove unused subplots
    for i in range(num_plots, len(axes)):
        fig.delaxes(axes[i])

    plt.tight_layout(rect=[0, 0, 1, 0.96])  # Adjust layout for suptitle
    plt.show()

# Plot histograms for each feature type
plot_histograms(df_train, binary_features, 'Binary')


# 1. Split Training Set and Validation Set

Splitting the training and validation set works as an early diagnostic towards the performance of the model we train. This is done before the preprocessing steps to **avoid data leakage inbetween the sets**. If you want to use k-fold cross-validation, split the data later and do the cleaning and preprocessing separately for each split.

Note: For training, you should use the data contained in the `train` folder given by the TA. The `test` data is only used for kaggle submission.

In [63]:
# Split training set and validation set here, store into variables train_set and val_set.
# Remember to also keep the original training set before splitting. This will come important later.
# train_set, val_set = ...

# 2. Data Cleaning and Preprocessing

This step is the first thing to be done once a Data Scientist have grasped a general knowledge of the data. Raw data is **seldom ready for training**, therefore steps need to be taken to clean and format the data for the Machine Learning model to interpret.

By performing data cleaning and preprocessing, you ensure that your dataset is ready for model training, leading to more accurate and reliable machine learning results. These steps are essential for transforming raw data into a format that machine learning algorithms can effectively learn from and make predictions.

We will give some common methods for you to try, but you only have to **at least implement one method for each process**. For each step that you will do, **please explain the reason why did you do that process. Write it in a markdown cell under the code cell you wrote.**

## A. Data Cleaning

**Data cleaning** is the crucial first step in preparing your dataset for machine learning. Raw data collected from various sources is often messy and may contain errors, missing values, and inconsistencies. Data cleaning involves the following steps:

1. **Handling Missing Data:** Identify and address missing values in the dataset. This can include imputing missing values, removing rows or columns with excessive missing data, or using more advanced techniques like interpolation.

2. **Dealing with Outliers:** Identify and handle outliers, which are data points significantly different from the rest of the dataset. Outliers can be removed or transformed to improve model performance.

3. **Data Validation:** Check for data integrity and consistency. Ensure that data types are correct, categorical variables have consistent labels, and numerical values fall within expected ranges.

4. **Removing Duplicates:** Identify and remove duplicate rows, as they can skew the model's training process and evaluation metrics.

5. **Feature Engineering**: Create new features or modify existing ones to extract relevant information. This step can involve scaling, normalizing, or encoding features for better model interpretability.

### I. Handling Missing Data

Missing data can adversely affect the performance and accuracy of machine learning models. There are several strategies to handle missing data in machine learning:

1. **Data Imputation:**

    a. **Mean, Median, or Mode Imputation:** For numerical features, you can replace missing values with the mean, median, or mode of the non-missing values in the same feature. This method is simple and often effective when data is missing at random.

    b. **Constant Value Imputation:** You can replace missing values with a predefined constant value (e.g., 0) if it makes sense for your dataset and problem.

    c. **Imputation Using Predictive Models:** More advanced techniques involve using predictive models to estimate missing values. For example, you can train a regression model to predict missing numerical values or a classification model to predict missing categorical values.

2. **Deletion of Missing Data:**

    a. **Listwise Deletion:** In cases where the amount of missing data is relatively small, you can simply remove rows with missing values from your dataset. However, this approach can lead to a loss of valuable information.

    b. **Column (Feature) Deletion:** If a feature has a large number of missing values and is not critical for your analysis, you can consider removing that feature altogether.

3. **Domain-Specific Strategies:**

    a. **Domain Knowledge:** In some cases, domain knowledge can guide the imputation process. For example, if you know that missing values are related to a specific condition, you can impute them accordingly.

4. **Imputation Libraries:**

    a. **Scikit-Learn:** Scikit-Learn provides a `SimpleImputer` class that can handle basic imputation strategies like mean, median, and mode imputation.

    b. **Fancyimpute:** Fancyimpute is a Python library that offers more advanced imputation techniques, including matrix factorization, k-nearest neighbors, and deep learning-based methods.

The choice of imputation method should be guided by the nature of your data, the amount of missing data, the problem you are trying to solve, and the assumptions you are willing to make.

In [ ]:

def fill_missing_values(data):
  for feature in binary_features:
    # Isi NA dengan -1 terlebih dahulu
    data[feature] = df_train[feature].fillna(-1)
    # Konversi nilai menjadi integer
    data[feature] = df_train[feature].astype(int)

  data[categorical_features] = data[categorical_features].fillna("Unknown")

  for feature in numerical_features:
    # 1. Akhirnya diisi median AWKOAWKOAW
    data[feature] = data[feature].fillna(data[feature].median())

    # 2. Metode Iterative Imputer ga kelar kelar 🗿🙏🙏
      # imputer = IterativeImputer(max_iter=10, random_state=42)
      # data[numerical_features] = imputer.fit_transform(data[numerical_features])

    # 3. Metode Randomforest ga kelar kelar 🗿🙏🙏
      # train_data = data[data[col].notna()]
      # predict_data = data[data[col].isna()]

      # X_train = train_data.drop(columns=[col])
      # y_train = train_data[col]
      # X_train = pd.get_dummies(X_train, drop_first=True)

      # X_predict = predict_data.drop(columns=[col])
      # X_predict = pd.get_dummies(X_predict, drop_first=True)
      # X_predict = X_predict.reindex(columns=X_train.columns, fill_value=0)

      # model = RandomForestRegressor(random_state=42, n_estimators=100)
      # model.fit(X_train, y_train)
      # data.loc[data[col].isna(), col] = model.predict(X_predict)


  return data

df_train = fill_missing_values(df_train)

# Diprint lagi biar keliatan before afternya
display_feature_summaries(df_train, categorical_features, numerical_features, binary_features)

### II. Dealing with Outliers

Outliers are data points that significantly differ from the majority of the data. They can be unusually high or low values that do not fit the pattern of the rest of the dataset. Outliers can significantly impact model performance, so it is important to handle them properly.

Some methods to handle outliers:
1. **Imputation**: Replace with mean, median, or a boundary value.
2. **Clipping**: Cap values to upper and lower limits.
3. **Transformation**: Use log, square root, or power transformations to reduce their influence.
4. **Model-Based**: Use algorithms robust to outliers (e.g., tree-based models, Huber regression).

In [ ]:
def transform_outliers(data, numerical_features):
    #1. Outlier di numerik di-cap ke batas atas dan bawah quartil
    for feature in numerical_features:
        Q1 = data[feature].quantile(0.25)
        Q3 = data[feature].quantile(0.75)

        IQR = Q3 - Q1

        data[feature] = data[feature].apply(lambda x: max(x, Q1 - 1.5 * IQR) if x < (Q1 - 1.5 * IQR) else x)
        data[feature] = data[feature].apply(lambda x: min(x, Q3 + 1.5 * IQR) if x > (Q3 + 1.5 * IQR) else x)

    #2. Untuk kategorikal udah dicek secara EDA (kasat mata) tu aman

    #3. Untuk binary coy ada yg eror
    #Katanya is_ftp_login binary tapi kok ada juga yg nilainya 2 dan 4?
    data['is_ftp_login'] = data['is_ftp_login'].apply(lambda x: x if x in [0, 1] else -1)
    return data

df_train = transform_outliers(df_train, numerical_features)


#Print lagi biar keliatan before afternya
display_feature_summaries(df_train, categorical_features, numerical_features, binary_features)

### III. Remove Duplicates
Handling duplicate values is crucial because they can compromise data integrity, leading to inaccurate analysis and insights. Duplicate entries can bias machine learning models, causing overfitting and reducing their ability to generalize to new data. They also inflate the dataset size unnecessarily, increasing computational costs and processing times. Additionally, duplicates can distort statistical measures and lead to inconsistencies, ultimately affecting the reliability of data-driven decisions and reporting. Ensuring data quality by removing duplicates is essential for accurate, efficient, and consistent analysis.

In [ ]:
# Write your code here

duplicate_count = df_train.duplicated().sum()
markdown_content = "### Duplicates\n"
markdown_content += f"* Total duplicates: {duplicate_count}\n"
display(Markdown(markdown_content))

# View the duplicated rows
if duplicate_count > 0:
    duplicates = df_train[df_train.duplicated(keep=False)]
    markdown_content = "#### Duplicate rows\n"
    display(Markdown(markdown_content))
    display(duplicates)

    df_train.drop_duplicates(inplace=True)
    markdown_content = f"* Duplicates removed.\n"
    display(Markdown(markdown_content))
else:
    markdown_content = f"* No duplicates found.\n"
    display(Markdown(markdown_content))

### IV. Feature Engineering

**Feature engineering** involves creating new features (input variables) or transforming existing ones to improve the performance of machine learning models. Feature engineering aims to enhance the model's ability to learn patterns and make accurate predictions from the data. It's often said that "good features make good models."

1. **Feature Selection:** Feature engineering can involve selecting the most relevant and informative features from the dataset. Removing irrelevant or redundant features not only simplifies the model but also reduces the risk of overfitting.

2. **Creating New Features:** Sometimes, the existing features may not capture the underlying patterns effectively. In such cases, engineers create new features that provide additional information. For example:
   
   - **Polynomial Features:** Engineers may create new features by taking the square, cube, or other higher-order terms of existing numerical features. This can help capture nonlinear relationships.
   
   - **Interaction Features:** Interaction features are created by combining two or more existing features. For example, if you have features "length" and "width," you can create an "area" feature by multiplying them.

3. **Binning or Discretization:** Continuous numerical features can be divided into bins or categories. For instance, age values can be grouped into bins like "child," "adult," and "senior."

4. **Domain-Specific Feature Engineering:** Depending on the domain and problem, engineers may create domain-specific features. For example, in fraud detection, features related to transaction history and user behavior may be engineered to identify anomalies.

Feature engineering is both a creative and iterative process. It requires a deep understanding of the data, domain knowledge, and experimentation to determine which features will enhance the model's predictive power.

In [67]:
# 3. Data Binning
def bin_features(df):
    # Binning Dur udah pasti maksimalnya 60 detik
    df['dur_bins'] = pd.cut(df['dur'], bins=[0, 20, 40, 60], 
                            labels=['short', 'medium', 'long'], include_lowest=True)

    # Binning'sload' and 'dload' pake quantiles aja
    # Distribusi datanya lumayan oke
    df['sload_bins'] = pd.qcut(df['sload'], q=4, labels=['very_low', 'low', 'high', 'very_high'])
    df['dload_bins'] = pd.qcut(df['dload'], q=4, labels=['very_low', 'low', 'high', 'very_high'])

    # Binning sjit' and 'djit' jomplang banget banyak nolnya jadi dibagi dua dari rata-rata aja
    # Kalo dibagi dua dari modus/median bakal rada bias (masa yg 0 juga masuk ke high?)
    df['sjit_bins'] = pd.cut(df['sjit'], bins=[-float('inf'), df['sjit'].mean(), float('inf')], 
                             labels=['low', 'high'])
    df['djit_bins'] = pd.cut(df['djit'], bins=[-float('inf'), df['djit'].mean(), float('inf')], 
                             labels=['low', 'high'])

    # Binning for 'sinpkt' and 'dinpkt' pake quartiles
    df['sinpkt_bins'] = pd.qcut(df['sinpkt'], q=4, labels=['very_low', 'low', 'high', 'very_high'])
    df['dinpkt_bins'] = pd.qcut(df['dinpkt'], q=4, labels=['very_low', 'low', 'high', 'very_high'])

    # Binning for 'tcprtt', 'synack', 'ackdat' bikin custom aja soalnya distribusinya unik
    df['tcprtt_bins'] = pd.cut(df['tcprtt'], bins=[0, 0.1, 0.5, float('inf')], 
                               labels=['very_short', 'short', 'long'])
    df['synack_bins'] = pd.cut(df['synack'], bins=[0, 0.05, 0.2, float('inf')], 
                               labels=['very_short', 'short', 'long'])
    df['ackdat_bins'] = pd.cut(df['ackdat'], bins=[0, 0.05, 0.2, float('inf')], 
                               labels=['very_short', 'short', 'long'])

    # Binning for 'smean' and 'dmean'pake quartiles
    # Sebab distribusinya lumayan oke
    df['smean_bins'] = pd.qcut(df['smean'], q=4, labels=['very_low', 'low', 'high', 'very_high'])
    df['dmean_bins'] = pd.qcut(df['dmean'], q=4, labels=['very_low', 'low', 'high', 'very_high'])

    return df

# Apply the function to your dataset
df_train = bin_features(df_train)



## B. Data Preprocessing

**Data preprocessing** is a broader step that encompasses both data cleaning and additional transformations to make the data suitable for machine learning algorithms. Its primary goals are:

1. **Feature Scaling:** Ensure that numerical features have similar scales. Common techniques include Min-Max scaling (scaling to a specific range) or standardization (mean-centered, unit variance).

2. **Encoding Categorical Variables:** Machine learning models typically work with numerical data, so categorical variables need to be encoded. This can be done using one-hot encoding, label encoding, or more advanced methods like target encoding.

3. **Handling Imbalanced Classes:** If dealing with imbalanced classes in a binary classification task, apply techniques such as oversampling, undersampling, or using different evaluation metrics to address class imbalance.

4. **Dimensionality Reduction:** Reduce the number of features using techniques like Principal Component Analysis (PCA) or feature selection to simplify the model and potentially improve its performance.

5. **Normalization:** Normalize data to achieve a standard distribution. This is particularly important for algorithms that assume normally distributed data.

### Notes on Preprocessing processes

It is advised to create functions or classes that have the same/similar type of inputs and outputs, so you can add, remove, or swap the order of the processes easily. You can implement the functions or classes by yourself

or

use `sklearn` library. To create a new preprocessing component in `sklearn`, implement a corresponding class that includes:
1. Inheritance to `BaseEstimator` and `TransformerMixin`
2. The method `fit`
3. The method `transform`

In [68]:
# Example

# from sklearn.base import BaseEstimator, TransformerMixin

# class FeatureEncoder(BaseEstimator, TransformerMixin):

#     def fit(self, X, y=None):

#         # Fit the encoder here

#         return self

#     def transform(self, X):
#         X_encoded = X.copy()

#         # Encode the categorical variables here

#         return X_encoded

### I. Feature Scaling

**Feature scaling** is a preprocessing technique used in machine learning to standardize the range of independent variables or features of data. The primary goal of feature scaling is to ensure that all features contribute equally to the training process and that machine learning algorithms can work effectively with the data.

Here are the main reasons why feature scaling is important:

1. **Algorithm Sensitivity:** Many machine learning algorithms are sensitive to the scale of input features. If the scales of features are significantly different, some algorithms may perform poorly or take much longer to converge.

2. **Distance-Based Algorithms:** Algorithms that rely on distances or similarities between data points, such as k-nearest neighbors (KNN) and support vector machines (SVM), can be influenced by feature scales. Features with larger scales may dominate the distance calculations.

3. **Regularization:** Regularization techniques, like L1 (Lasso) and L2 (Ridge) regularization, add penalty terms based on feature coefficients. Scaling ensures that all features are treated equally in the regularization process.

Common methods for feature scaling include:

1. **Min-Max Scaling (Normalization):** This method scales features to a specific range, typically [0, 1]. It's done using the following formula:

   $$X' = \frac{X - X_{min}}{X_{max} - X_{min}}$$

   - Here, $X$ is the original feature value, $X_{min}$ is the minimum value of the feature, and $X_{max}$ is the maximum value of the feature.  
<br />
<br />
2. **Standardization (Z-score Scaling):** This method scales features to have a mean (average) of 0 and a standard deviation of 1. It's done using the following formula:

   $$X' = \frac{X - \mu}{\sigma}$$

   - $X$ is the original feature value, $\mu$ is the mean of the feature, and $\sigma$ is the standard deviation of the feature.  
<br />
<br />
3. **Robust Scaling:** Robust scaling is a method that scales features to the interquartile range (IQR) and is less affected by outliers. It's calculated as:

   $$X' = \frac{X - Q1}{Q3 - Q1}$$

   - $X$ is the original feature value, $Q1$ is the first quartile (25th percentile), and $Q3$ is the third quartile (75th percentile) of the feature.  
<br />
<br />
4. **Log Transformation:** In cases where data is highly skewed or has a heavy-tailed distribution, taking the logarithm of the feature values can help stabilize the variance and improve scaling.

The choice of scaling method depends on the characteristics of your data and the requirements of your machine learning algorithm. **Min-max scaling and standardization are the most commonly used techniques and work well for many datasets.**

Scaling should be applied separately to each training and test set to prevent data leakage from the test set into the training set. Additionally, **some algorithms may not require feature scaling, particularly tree-based models.**

In [69]:
# Write your code here

### II. Feature Encoding

**Feature encoding**, also known as **categorical encoding**, is the process of converting categorical data (non-numeric data) into a numerical format so that it can be used as input for machine learning algorithms. Most machine learning models require numerical data for training and prediction, so feature encoding is a critical step in data preprocessing.

Categorical data can take various forms, including:

1. **Nominal Data:** Categories with no intrinsic order, like colors or country names.  

2. **Ordinal Data:** Categories with a meaningful order but not necessarily equidistant, like education levels (e.g., "high school," "bachelor's," "master's").

There are several common methods for encoding categorical data:

1. **Label Encoding:**

   - Label encoding assigns a unique integer to each category in a feature.
   - It's suitable for ordinal data where there's a clear order among categories.
   - For example, if you have an "education" feature with values "high school," "bachelor's," and "master's," you can encode them as 0, 1, and 2, respectively.
<br />
<br />
2. **One-Hot Encoding:**

   - One-hot encoding creates a binary (0 or 1) column for each category in a nominal feature.
   - It's suitable for nominal data where there's no inherent order among categories.
   - Each category becomes a new feature, and the presence (1) or absence (0) of a category is indicated for each row.
<br />
<br />
3. **Target Encoding (Mean Encoding):**

   - Target encoding replaces each category with the mean of the target variable for that category.
   - It's often used for classification problems.

In [ ]:
# Untuk binary_features sebab udah 0,1, jadi gausah diapa-apain lagi
def encode_binary_features(data, binary_features):
    for feature in binary_features:
        data[feature] = data[feature].astype(int)

# Untuk kategori nominal, maka harus pake one hot
# Kebetulan semua kategorikal di data adalah nominal
nominal_features = ['proto', 'state', 'service']
def encode_categorical_features(data, nominal_features):
    one_hot_encoder = OneHotEncoder(sparse_output=False)
    categorical_encoded = one_hot_encoder.fit_transform(data[nominal_features])
    categorical_encoded_data = pd.DataFrame(categorical_encoded, columns=one_hot_encoder.get_feature_names_out(categorical_features))

    # Masukin hasil encoding ke data asli
    data = pd.concat([data.reset_index(drop=True), categorical_encoded_data.reset_index(drop=True)], axis=1)
    data.drop(columns=categorical_features, inplace=True)

# Target feature juga perlu di-encode supaya bisa masuk ke algoritma
def encode_target_feature(data):
    label_encoder = LabelEncoder()
    data['attack_cat_encoded'] = label_encoder.fit_transform(data['attack_cat'].fillna('Unknown'))

    # Drop yang asli
    data.drop(columns=['attack_cat'], inplace=True)

# Encoding

encode_binary_features(df_train, binary_features)
encode_categorical_features(df_train, nominal_features)
encode_target_feature(df_train)
print("Encoding complete!")


### III. Handling Imbalanced Dataset

**Handling imbalanced datasets** is important because imbalanced data can lead to several issues that negatively impact the performance and reliability of machine learning models. Here are some key reasons:

1. **Biased Model Performance**:

 - Models trained on imbalanced data tend to be biased towards the majority class, leading to poor performance on the minority class. This can result in misleading accuracy metrics.

2. **Misleading Accuracy**:

 - High overall accuracy can be misleading in imbalanced datasets. For example, if 95% of the data belongs to one class, a model that always predicts the majority class will have 95% accuracy but will fail to identify the minority class.

3. **Poor Generalization**:

 - Models trained on imbalanced data may not generalize well to new, unseen data, especially if the minority class is underrepresented.


Some methods to handle imbalanced datasets:
1. **Resampling Methods**:

 - Oversampling: Increase the number of instances in the minority class by duplicating or generating synthetic samples (e.g., SMOTE).
 - Undersampling: Reduce the number of instances in the majority class to balance the dataset.

2. **Evaluation Metrics**:

 - Use appropriate evaluation metrics such as precision, recall, F1-score, ROC-AUC, and confusion matrix instead of accuracy to better assess model performance on imbalanced data.

3. **Algorithmic Approaches**:

 - Use algorithms that are designed to handle imbalanced data, such as decision trees, random forests, or ensemble methods.
 - Adjust class weights in algorithms to give more importance to the minority class.

In [105]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline

# TODO: Unfinished code

# The dataset must be in numerical form before applying the resampling techniques

# Write your code here

# Display df_train.head for categorical_features only
categorical_features.remove('attack_cat')
# display(df_train[categorical_features].head())
# display(df_train['attack_cat_encoded'].head())

# Visualize the balance comparation of the target feature (attack_cat feature is categorical)
sn.countplot(x='attack_cat_encoded', data=df_train.drop(columns=categorical_features))
plt.title('Distribution of attack_cat')
plt.xticks(rotation=45)
plt.show()

categorical_features.append('attack_cat_encoded')

# Define the target and features
X = df_train.drop(columns=categorical_features)

# Drop categorical features from X
# print(categorical_features)
# X = X.drop(columns=categorical_features)

y = df_train['attack_cat_encoded']

# Define the undersampling and oversampling strategy
undersample = RandomUnderSampler(sampling_strategy='majority')
oversample = SMOTE(sampling_strategy='minority')

# Create a pipeline with undersampling and oversampling
pipeline = Pipeline(steps=[('u', undersample), ('o', oversample)])

# Apply the pipeline to the data
X_resampled, y_resampled = pipeline.fit_resample(X, y)

# Combine the resampled features and target into a new DataFrame
df_resampled = pd.concat([X_resampled, y_resampled], axis=1)

# Display the new distribution of the target feature
sn.countplot(x='attack_cat_encoded', data=df_resampled)
plt.title('Distribution of attack_cat after Resampling')
plt.xticks(rotation=45)
plt.show()

### IV. Data Normalization

Data normalization is used to achieve a standard distribution. Without normalization, models or processes that rely on the assumption of normality may not work correctly. Normalization helps reduce the magnitude effect and ensures numerical stability during optimization.

In [106]:
# Write your code here

### V. Dimensionality Reduction

Dimensionality reduction is a technique used in data preprocessing to reduce the number of input features (dimensions) in a dataset while retaining as much important information as possible. It is essential when dealing with high-dimensional data, where too many features can cause problems like increased computational costs, overfitting, and difficulty in visualization. Reducing dimensions simplifies the data, making it easier to analyze and improving the performance of machine learning models.

One of the main approaches to dimensionality reduction is feature extraction. Feature extraction creates new, smaller sets of features that capture the essence of the original data. Common techniques include:

1. **Principal Component Analysis (PCA)**: Converts correlated features into a smaller number of uncorrelated "principal components."
2. **t-SNE (t-Distributed Stochastic Neighbor Embedding)**: A visualization-focused method to project high-dimensional data into 2D or 3D spaces.
3. **Autoencoders**: Neural networks that learn compressed representations of the data.

In [107]:
# Write your code here

# 3. Compile Preprocessing Pipeline

All of the preprocessing classes or functions defined earlier will be compiled in this step.

If you use sklearn to create preprocessing classes, you can list your preprocessing classes in the Pipeline object sequentially, and then fit and transform your data.

In [108]:
# from sklearn.pipeline import Pipeline

# # Note: You can add or delete preprocessing components from this pipeline

# pipe = Pipeline([("imputer", FeatureImputer()),
#                  ("featurecreator", FeatureCreator()),
#                  ("scaler", FeatureScaler()),
#                  ("encoder", FeatureEncoder())])

# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

In [109]:
# # Your code should work up until this point
# train_set = pipe.fit_transform(train_set)
# val_set = pipe.transform(val_set)

or create your own here

In [110]:
# Write your code here

# 4. Modeling and Validation

Modelling is the process of building your own machine learning models to solve specific problems, or in this assignment context, predicting the target feature `attack_cat`. Validation is the process of evaluating your trained model using the validation set or cross-validation method and providing some metrics that can help you decide what to do in the next iteration of development.

## A. KNN

In [111]:
# Type your code here

## B. Naive Bayes

In [112]:
# Type your code here

## C. ID3

In [113]:
# Type your code here

## D. Improvements (Optional)

- **Visualize the model evaluation result**

This will help you to understand the details more clearly about your model's performance. From the visualization, you can see clearly if your model is leaning towards a class than the others. (Hint: confusion matrix, ROC-AUC curve, etc.)

- **Explore the hyperparameters of your models**

Each models have their own hyperparameters. And each of the hyperparameter have different effects on the model behaviour. You can optimize the model performance by finding the good set of hyperparameters through a process called **hyperparameter tuning**. (Hint: Grid search, random search, bayesian optimization)

- **Cross-validation**

Cross-validation is a critical technique in machine learning and data science for evaluating and validating the performance of predictive models. It provides a more **robust** and **reliable** evaluation method compared to a hold-out (single train-test set) validation. Though, it requires more time and computing power because of how cross-validation works. (Hint: k-fold cross-validation, stratified k-fold cross-validation, etc.)

In [114]:
# Type your code here

## E. Submission
To predict the test set target feature and submit the results to the kaggle competition platform, do the following:
1. Create a new pipeline instance identical to the first in Data Preprocessing
2. With the pipeline, apply `fit_transform` to the original training set before splitting, then only apply `transform` to the test set.
3. Retrain the model on the preprocessed training set
4. Predict the test set
5. Make sure the submission contains the `id` and `attack_cat` column.

In [115]:
# Type your code here

# 6. Error Analysis

Based on all the process you have done until the modeling and evaluation step, write an analysis to support each steps you have taken to solve this problem. Write the analysis using the markdown block. Some questions that may help you in writing the analysis:

- Does my model perform better in predicting one class than the other? If so, why is that?
- To each models I have tried, which performs the best and what could be the reason?
- Is it better for me to impute or drop the missing data? Why?
- Does feature scaling help improve my model performance?
- etc...

`Provide your analysis here`